In [1]:
import os
import torch
from doctr.datasets import VOCABS
from doctr.models import vitstr_small, ocr_predictor
from doctr.io import DocumentFile
from PIL import Image
from tempfile import TemporaryDirectory
from ocrmypdf.hocrtransform import HocrTransform
from PyPDF2 import PdfMerger

/mnt/netapp2/Store_uni/home/ulc/cursos/curso341/my_env310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Fonction pour vérifier si CUDA est disponible
def check_cuda():
    """Vérifie si CUDA est disponible pour l'accélération GPU."""
    return torch.cuda.is_available()

In [3]:
# Fonction pour initialiser le modèle OCR avec le vocabulaire
def initialize_ocr_model(vocab="english", model_path="vitstr_small_20250427-124227.pt"):
    """Initialise le modèle OCR avec un vocabulaire donné et un chemin vers le modèle."""
    # Chargement du vocabulaire
    vocab = VOCABS[vocab]

    # Initialisation du modèle
    reco_model = vitstr_small(pretrained=False, pretrained_backbone=False, vocab=vocab)

    # Chargement des poids du modèle
    reco_params = torch.load(model_path, map_location="cpu")
    reco_model.load_state_dict(reco_params)
    
    # Initialisation du prédicteur OCR
    predictor = ocr_predictor(det_arch="db_resnet50", reco_arch=reco_model, pretrained=True)

    # Déplacer sur GPU si disponible
    if check_cuda():
        predictor.cuda().half()

    return predictor


In [4]:
# Fonction pour traiter un document PDF et obtenir les résultats OCR
def process_pdf(file_path, predictor):
    """Charge et traite un fichier PDF pour en extraire les résultats OCR."""
    doc = DocumentFile.from_pdf(file_path)
    result = predictor(doc)
    return result, doc

In [5]:
# Fonction pour enregistrer les résultats OCR en un seul fichier PDF
def save_ocr_results(result, doc, output_pdf_path):
    """Enregistre les résultats OCR sous forme d'un fichier PDF/A avec le même nom que le fichier d'origine."""
    # Exporter les résultats OCR au format XML
    xml_outputs = result.export_as_xml()

    # Créer un répertoire temporaire 'temp' s'il n'existe pas
    temp_dir = "temp"
    os.makedirs(temp_dir, exist_ok=True)

    # Créer une liste pour les chemins des fichiers PDF
    pdf_paths = []

    # Sauvegarder les images et fichiers XML pour chaque page
    for i, (xml, img) in enumerate(zip(xml_outputs, doc)):
        # Sauvegarder l'image sous forme de fichier JPEG
        img_path = os.path.join(temp_dir, f"{i}.jpg")
        Image.fromarray(img).save(img_path)

        # Sauvegarder le fichier XML
        xml_path = os.path.join(temp_dir, f"{i}.xml")
        with open(xml_path, "w") as f:
            f.write(xml[0].decode())  # xml[0] contient la chaîne XML en bytes

        # Initialiser le transformateur hOCR pour convertir en PDF/A pour chaque page
        hocr = HocrTransform(hocr_filename=xml_path, dpi=300)
        page_pdf_path = os.path.join(temp_dir, f"{i}_page.pdf")
        
        # Sauvegarder chaque page en PDF/A
        hocr.to_pdf(out_filename=page_pdf_path, image_filename=img_path)

        # Ajouter le fichier PDF de la page à la liste
        pdf_paths.append(page_pdf_path)

    # Créer un fichier PDF final en concaténant tous les fichiers PDF de chaque page
    merger = PdfMerger()

    # Ajouter chaque fichier PDF de page à l'instance de PdfMerger
    for pdf in pdf_paths:
        merger.append(pdf)

    # Sauvegarder le fichier PDF final dans le répertoire 'output'
    merger.write(output_pdf_path)
    merger.close()

    # Supprimer le contenu du répertoire temporaire
    for pdf_path in pdf_paths:
        os.remove(pdf_path)
    for file in os.listdir(temp_dir):
        file_path = os.path.join(temp_dir, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
    os.rmdir(temp_dir)

    print(f"Le fichier PDF/A final a été sauvegardé sous '{output_pdf_path}'.")

In [6]:
# Fonction pour traiter tous les fichiers PDF dans un répertoire d'entrée
def process_pdfs_in_directory(input_dir, output_dir, predictor):
    """Traite tous les fichiers PDF dans un répertoire donné et les enregistre sous forme de PDF/A."""
    # Liste tous les fichiers dans le répertoire d'entrée
    for filename in os.listdir(input_dir):
        file_path = os.path.join(input_dir, filename)

        # Vérifie si c'est un fichier PDF
        if filename.lower().endswith('.pdf'):
            print(f"Traitement du fichier: {filename}")
            # Traite le PDF et obtient les résultats OCR
            result, doc = process_pdf(file_path, predictor)

            # Créer le nom de fichier de sortie avec le même nom que le fichier d'origine
            output_pdf_path = os.path.join(output_dir, filename)

            # Sauvegarder les résultats sous forme de PDF/A avec le même nom
            save_ocr_results(result, doc, output_pdf_path)
        else:
            print(f"Le fichier {filename} n'est pas un PDF, il sera ignoré.")


In [7]:
# Fonction principale pour initialiser et lancer le processus
def main(input_dir, output_dir, vocab="english", model_path="vitstr_small_20250427-124227.pt"):
    """Fonction principale pour initialiser le modèle OCR et traiter les PDF dans un répertoire."""
    print("Vérification de la disponibilité de CUDA:", check_cuda())
    os.makedirs(output_dir, exist_ok=True)
    # Initialiser le modèle OCR
    predictor = initialize_ocr_model(vocab, model_path)

    # Traiter tous les fichiers PDF dans le répertoire d'entrée
    process_pdfs_in_directory(input_dir, output_dir, predictor)

In [8]:
# Exemple d'appel de la fonction principale
if __name__ == "__main__":
    input_dir = "dataset_legacy_SAVE"  # Répertoire des fichiers PDF à traiter
    output_dir = "output"         # Répertoire de sortie pour les PDF/A traités
    vocab = "english"  # Vocabulaire à utiliser
    model_path = "vitstr_small_20250427-124227.pt"  # Chemin vers le modèle OCR
    main(input_dir, output_dir)

Vérification de la disponibilité de CUDA: False
Traitement du fichier: History_of_Business-_1892-_handwritten_notes_(1).pdf
Le fichier PDF/A final a été sauvegardé sous 'output/History_of_Business-_1892-_handwritten_notes_(1).pdf'.
Traitement du fichier: Lettre_Feudataire_abbaye_de_Mercoire.pdf
Le fichier PDF/A final a été sauvegardé sous 'output/Lettre_Feudataire_abbaye_de_Mercoire.pdf'.
Traitement du fichier: COI-handwritten notes.pdf
Le fichier PDF/A final a été sauvegardé sous 'output/COI-handwritten notes.pdf'.
Traitement du fichier: Cash-Payment-Receipt-Template.pdf
Le fichier PDF/A final a été sauvegardé sous 'output/Cash-Payment-Receipt-Template.pdf'.
Traitement du fichier: Cash-Payment-Receipt-Template copy.pdf
Le fichier PDF/A final a été sauvegardé sous 'output/Cash-Payment-Receipt-Template copy.pdf'.
